## Assignment 5 – Scraping

Aviv Yefet 
ID: 208495267

In [48]:
#installing pandas libraries
!pip install pandas-datareader  
# for reading files
!pip install --upgrade html5lib==1.0b8 
# for reading html

#There is a bug in the latest version of html5lib so install an earlier version
#Restart kernel after installing html5lib

  Using cached html5lib-1.0b8-py3-none-any.whl
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.1
    Uninstalling html5lib-1.1:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\html5lib-1.1.dist-info\\AUTHORS.rst'
Consider using the `--user` option or check the permissions.



In [49]:
import pandas as pd #pandas library
from pandas_datareader import data #data readers (google, html, etc.)
#The following line ensures that graphs are rendered in the notebook (and not in another window)
%matplotlib inline 
from bs4 import BeautifulSoup
import numpy as np
import requests
import json 

In [97]:
path = "C:\\Users\\user\\Dropbox\\DS MyNotebooks\\"
fileName = "destinations_LP_crawler_Ex5.csv"

df = pd.read_csv( path + fileName ,encoding='ISO-8859-1')

df #examine our data

,city,city_LP
0,Antalya,mediterranean-coast/antalya
1,Athens,Athens
2,Ayia-napa,the-republic-of-cyprus/Agia-napa
3,Bangkok,Bangkok
4,Barcelona,Barcelona
...,...,...
71,Venice,Venice
72,Vienna,Vienna
73,Warsaw,Warsaw
74,Zakynthos,Zakynthos


In [52]:
# LP_destinations data-ftame will contain the results

LP_destinations = pd.DataFrame()

LP_destinations['City'] = df['city']
LP_destinations['lp_city'] = df['city_LP']


In [54]:
# Let's examine our data
# There is a problem with 'Krakï¿' city
LP_destinations['City'].unique()


array(['Antalya', 'Athens', 'Ayia-napa', 'Bangkok', 'Barcelona', 'Basel',
       'Batumi', 'Beijing', 'Berlin', 'Boston', 'Bratislava', 'Brussels',
       'Bucharest', 'Budapest', 'Burgas', 'Cancun', 'Catania',
       'Chiang-mai', 'Cologne', 'Corfu', 'Costa-Brava', 'Crete', 'Krakï¿',
       'Delhi', 'Dresden', 'Dublin', 'Dubrovnik', 'Dusseldorf',
       'Florence', 'Frankfurt', 'Geneva', 'Guangzhou', 'Hamburg',
       'Innsbruck', 'Istanbul', 'Kos', 'Ko-samui', 'Kyiv ukrain',
       'LarnaKa', 'Las-Vegas', 'Limassol', 'Lisbon', 'Ljubljana',
       'Los Angeles', 'Madrid', 'Manila', 'Marseille', 'Mexico-City',
       'Miami', 'Milan', 'Moscow', 'mumbai-bombay', 'Munich', 'Mykonos',
       'Naples Italy', 'Nice', 'Odesa', 'Paris', 'Phuket', 'Rhodes',
       'Rome', 'Salzburg', 'San-Francisco', 'Shanghai', 'Sofia',
       'St-Petersburg Russia', 'Stuttgart', 'Taipei', 'Tbilisi',
       'Thessaloniki', 'Toronto', 'Venice', 'Vienna', 'Warsaw',
       'Zakynthos', 'Zurich'], dtype=object)

In [98]:
# fix krakow city
np.where(LP_destinations['City']=='Krakï¿') # index=22
LP_destinations['City'][22]='Krakow'

LP_destinations['City'].unique() #DONE


array(['Antalya', 'Athens', 'Ayia-napa', 'Bangkok', 'Barcelona', 'Basel',
       'Batumi', 'Beijing', 'Berlin', 'Boston', 'Bratislava', 'Brussels',
       'Bucharest', 'Budapest', 'Burgas', 'Cancun', 'Catania',
       'Chiang-mai', 'Cologne', 'Corfu', 'Costa-Brava', 'Crete', 'Krakow',
       'Delhi', 'Dresden', 'Dublin', 'Dubrovnik', 'Dusseldorf',
       'Florence', 'Frankfurt', 'Geneva', 'Guangzhou', 'Hamburg',
       'Innsbruck', 'Istanbul', 'Kos', 'Ko-samui', 'Kyiv ukrain',
       'LarnaKa', 'Las-Vegas', 'Limassol', 'Lisbon', 'Ljubljana',
       'Los Angeles', 'Madrid', 'Manila', 'Marseille', 'Mexico-City',
       'Miami', 'Milan', 'Moscow', 'mumbai-bombay', 'Munich', 'Mykonos',
       'Naples Italy', 'Nice', 'Odesa', 'Paris', 'Phuket', 'Rhodes',
       'Rome', 'Salzburg', 'San-Francisco', 'Shanghai', 'Sofia',
       'St-Petersburg Russia', 'Stuttgart', 'Taipei', 'Tbilisi',
       'Thessaloniki', 'Toronto', 'Venice', 'Vienna', 'Warsaw',
       'Zakynthos', 'Zurich'], dtype=object)

### Find the country for each city:

In [99]:
#this function find the country by geocoding servise

def find_country(input_city):
    api_key = 'ApiKey'
    address = input_city
    url = "https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address,api_key)
    try:  
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP ERROR")
        else:
            try:
                response_data = response.json()
                country = response_data['results'][0]['formatted_address'].split(',')[1].strip()
                return country
            except:
                return np.NaN
    except:
        print("Somthing went wrong with request.get")
           

In [100]:
LP_destinations['Country'] = LP_destinations['City'].apply(find_country)

In [102]:
# Let's examine our data
LP_destinations['Country'].unique()

array(['Turkey', 'Greece', 'Cyprus', 'Thailand', 'Spain', 'Switzerland',
       'Georgia', 'China', 'Germany', 'MA', 'Slovakia', 'Belgium',
       'Romania', 'Hungary', 'Bulgaria', 'Quintana Roo',
       'Metropolitan city of Catania', 'Mueang Chiang Mai District',
       'Girona', 'Poland', 'India', 'Ireland', 'Croatia',
       'Metropolitan City of Florence', 'Guangdong Province', 'Austria',
       nan, 'Ko Samui District', 'Ukraine', 'NV', 'Portugal', 'Slovenia',
       'CA', 'Metro Manila', 'France', 'CDMX', 'FL',
       'Metropolitan City of Milan', 'Russia', 'Maharashtra',
       'Mikonos 846 00', 'Metropolitan City of Naples', 'Odessa Oblast',
       'Mueang Phuket District', 'Metropolitan City of Rome', 'Taiwan',
       'ON', 'Metropolitan City of Venice'], dtype=object)

In [105]:
# Data cleaning

def fix_country(input_country):
    input_country=str(input_country)
    if input_country.startswith('Metropolitan City of') or input_country.startswith('Metropolitan city of'):
        input_country = input_country.split()[3]
        return input_country
    elif input_country.startswith('Mueang') or input_country.startswith('Ko Samui'):
        input_country='Thailand'
        return input_country
    elif input_country =='Mikonos 846 00':
        return 'Mikonos'
    elif input_country =='MA' or input_country=='NV' or input_country=='CA' or input_country == 'FL':
        return 'usa'
    elif input_country == 'Quintana Roo' or input_country == 'CDMX':
        return 'mexico'
    elif input_country =='Girona':
        return 'spain'
    elif input_country == 'Catania' or input_country == 'Florence' or input_country == "Milan" or input_country == 'Naples' or input_country == 'Rome' or input_country == 'Venice':
        return 'italy'
    elif input_country == 'Guangdong Province':
        return 'china'
    elif input_country =='nan' or input_country == 'Mikonos':
        return 'greece'
    elif input_country == 'Metro Manila':
        return 'philippines'
    elif input_country == 'Maharashtra':
        return 'india'
    elif input_country == 'Odessa Oblast':
        return 'ukraine'
    elif input_country == 'ON':
        return 'canada'
    elif input_country == 'None':
        return 'poland'
    else:
        return input_country

In [106]:
LP_destinations['Country']=LP_destinations['Country'].apply(fix_country)

In [107]:
LP_destinations['Country'].unique() #All countries have been fixed

array(['Turkey', 'Greece', 'Cyprus', 'Thailand', 'Spain', 'Switzerland',
       'Georgia', 'China', 'Germany', 'usa', 'Slovakia', 'Belgium',
       'Romania', 'Hungary', 'Bulgaria', 'mexico', 'italy', 'spain',
       'Poland', 'India', 'Ireland', 'Croatia', 'china', 'Austria',
       'greece', 'Ukraine', 'Portugal', 'Slovenia', 'philippines',
       'France', 'Russia', 'india', 'ukraine', 'Taiwan', 'canada'],
      dtype=object)

## find the description for each city

In [88]:
# Build url
LP_destinations['url']= "https://www.lonelyplanet.com/" + LP_destinations['Country'] + "/" + LP_destinations['lp_city']   

In [89]:
def find_description(url):
    try:
        response = requests.get(url)
        result_page = BeautifulSoup(response.content,'html.parser')
        section = result_page.find_all("section", class_="text-black-400")
        
        div_section = section[0].find_all('div',class_="readMore_content___5EuR relative overflow-hidden max-h-96 is-max wysiwyg")
        result = ""
        for tag in div_section[0]:
            result += tag.getText()
        return result
    except:
        return np.NaN

In [90]:
LP_destinations['Description'] = LP_destinations['url'].apply(find_description)
LP_destinations

,City,Country,lp_city,url,Description
0,Antalya,Turkey,mediterranean-coast/antalya,https://www.lonelyplanet.com/Turkey/mediterran...,Once seen simply as the gateway to the Turkish...
1,Athens,Greece,Athens,https://www.lonelyplanet.com/Greece/Athens,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,Cyprus,the-republic-of-cyprus/Agia-napa,https://www.lonelyplanet.com/Cyprus/the-republ...,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Thailand,Bangkok,https://www.lonelyplanet.com/Thailand/Bangkok,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Spain,Barcelona,https://www.lonelyplanet.com/Spain/Barcelona,Barcelona is an enchanting seaside city with b...
...,...,...,...,...,...
71,Venice,italy,Venice,https://www.lonelyplanet.com/italy/Venice,Imagine the audacity of building a city of mar...
72,Vienna,Austria,Vienna,https://www.lonelyplanet.com/Austria/Vienna,Baroque streetscapes and imperial palaces set ...
73,Warsaw,poland,Warsaw,https://www.lonelyplanet.com/poland/Warsaw,"A phoenix arisen from the ashes, Poland's capi..."
74,Zakynthos,Greece,Zakynthos,https://www.lonelyplanet.com/Greece/Zakynthos,"Zakynthos, also known by its Italian name Zant..."


In [108]:
LP_destinations.info()
# cheking if there is a Null value 
# there is no missing value 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   City         76 non-null     object
 1   Country      76 non-null     object
 2   lp_city      76 non-null     object
 3   url          76 non-null     object
 4   Description  76 non-null     object
dtypes: object(5)
memory usage: 3.1+ KB


In [124]:
LP_destinations=LP_destinations.drop(['lp_city', 'url'], axis=1)

In [132]:
# LP_destinations final data frame

LP_destinations

,City,Country,Description
0,Antalya,Turkey,Once seen simply as the gateway to the Turkish...
1,Athens,Greece,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,Cyprus,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Thailand,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Spain,Barcelona is an enchanting seaside city with b...
...,...,...,...
71,Venice,italy,Imagine the audacity of building a city of mar...
72,Vienna,Austria,Baroque streetscapes and imperial palaces set ...
73,Warsaw,Poland,"A phoenix arisen from the ashes, Poland's capi..."
74,Zakynthos,Greece,"Zakynthos, also known by its Italian name Zant..."


In [126]:
LP_destinations.to_csv(path + 'LP_destinations.csv',index = False)